In [1]:
import os
import IPython
import uuid
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.functions import mean, stddev, col, min, max , avg, skewness as skew, variance, sum, lit, round, length
from pyspark.sql.types import StringType, IntegerType
import math
import requests
import http.client
import json
from azure.identity import DefaultAzureCredential, ClientSecretCredential
import pandas as pd
from typing import Tuple
import re
from ygdra import *



In [2]:
print(printit())

Hello World


In [3]:
# spark already exists in Databricks.
# if launching from databricks-connect, we need get a new instance
try:
  # check if spark already exists
  spark
  print('spark already exists')
except NameError:
  # spark does not exists, create an instance
  spark = SparkSession.builder.getOrCreate()
  print('create a spark instance')


create a spark instance


In [4]:
customers = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("Customers.csv")

customers.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Birthdate: string (nullable = true)
 |-- Street: string (nullable = true)
 |-- PostalCode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- RowId: string (nullable = true)
 |-- ModifiedDate: string (nullable = true)



In [5]:
pdCustomers = customers.toPandas()

pdCustomers.head()

,ID,FirstName,LastName,Email,Birthdate,Street,PostalCode,Country,RowId,ModifiedDate
0,1,Galvin,Franks,diam@Nullaeuneque.net,1991-03-17,"P.O. Box 277, 2586 Nec Rd.",881240,Pitcairn Islands,3AAA3D94-10CA-C1DB-240E-BCB6E9D413CF,2021-03-31
1,2,Lucius,Shelton,luctus.ut.pellentesque@magnaUt.org,2003-07-23,"Ap #530-6075 Urna, Street",2921,Grenada,877E6185-9918-993D-E676-CBE6E0AF6467,2021-11-16
2,3,Taylor,Walters,purus.Duis.elementum@mi.org,2003-09-12,642-5534 Nunc Ave,30705,Indonesia,40A47836-48B6-451A-C263-68179927B95B,2021-02-19
3,4,Cara,Cooper,Pellentesque.ut.ipsum@liberoMorbiaccumsan.org,1984-06-19,"P.O. Box 109, 3683 Amet, Ave",01818,Saint Pierre and Miquelon,6B1880BD-91A1-0516-3460-1901FE0750B0,2021-09-18
4,5,Stephanie,Avery,ultricies@antedictum.ca,1992-09-16,567-6446 Ipsum Av.,474070,Monaco,79C30667-C768-E5FE-B1E4-BA5B2AB0C6A0,2021-05-22


In [7]:
pdCustomers[["LastName", "FirstName"]].describe()

,LastName,FirstName
count,200,200
unique,183,182
top,William,Cruz
freq,3,3


In [8]:
pdCustomers.describe()

,ID
count,200.000000
mean,100.500000
std,57.879185
min,1.000000
25%,50.750000
50%,100.500000
75%,150.250000
max,200.000000


In [16]:
pdCustomers.describe(include='all')

,ID,FirstName,LastName,Email,Birthdate,Street,PostalCode,Country,RowId,ModifiedDate
count,200.000000,200,200,200,200,200,200,200,200,200
unique,NaN,182,183,200,199,200,200,134,200,171
top,NaN,Cruz,William,Nunc.pulvinar@Maecenas.org,2003-09-12,Ap #254-5428 Cum Avenue,67765,New Caledonia,804F0088-A9D7-CD87-2219-C2D5D4A1BDB5,2020-11-29
freq,NaN,3,3,1,2,1,1,5,1,3
mean,100.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,57.879185,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,50.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,100.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,150.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
pdCustomers.describe(include='all').to_json()

'{"ID":{"count":200.0,"unique":null,"top":null,"freq":null,"mean":100.5,"std":57.879184514,"min":1.0,"25%":50.75,"50%":100.5,"75%":150.25,"max":200.0},"FirstName":{"count":200,"unique":182,"top":"Cruz","freq":3,"mean":null,"std":null,"min":null,"25%":null,"50%":null,"75%":null,"max":null},"LastName":{"count":200,"unique":183,"top":"William","freq":3,"mean":null,"std":null,"min":null,"25%":null,"50%":null,"75%":null,"max":null},"Email":{"count":200,"unique":200,"top":"Nunc.pulvinar@Maecenas.org","freq":1,"mean":null,"std":null,"min":null,"25%":null,"50%":null,"75%":null,"max":null},"Birthdate":{"count":200,"unique":199,"top":"2003-09-12","freq":2,"mean":null,"std":null,"min":null,"25%":null,"50%":null,"75%":null,"max":null},"Street":{"count":200,"unique":200,"top":"Ap #254-5428 Cum Avenue","freq":1,"mean":null,"std":null,"min":null,"25%":null,"50%":null,"75%":null,"max":null},"PostalCode":{"count":200,"unique":200,"top":"67765","freq":1,"mean":null,"std":null,"min":null,"25%":null,"50%"

In [33]:
# Viewing last 3 lines
pdCustomers.tail(3)

# Viewing first 3 lines
# pdCustomers.head(3)

,ID,FirstName,LastName,Email,Birthdate,Street,PostalCode,Country,RowId,ModifiedDate
197,198,Kitra,Hubbard,et@lectusa.org,1988-02-08,Ap #221-5327 Sed Rd.,9384,New Caledonia,F74BFBF0-1FF3-5C9F-4F9B-5BA41434DE61,2020-06-28
198,199,Colby,Quinn,cursus.Integer@Maecenasmi.co.uk,1979-06-28,435-6566 Eleifend Rd.,L1C 9B3,French Guiana,D318472C-9A8E-CF79-BDD3-2999C0EAE4F7,2020-10-25
199,200,Declan,Decker,Cum@penatibus.co.uk,1981-08-26,870-6125 Ornare. Av.,L6 0ZR,Liberia,BEC6C4BC-BA38-183B-8DA2-E49D4D0EF179,2020-06-18


In [21]:
ages = pd.Series([22, 35, 58], name="Age")

ages.describe()

count     3.000000
mean     38.333333
std      18.230012
min      22.000000
25%      28.500000
50%      35.000000
75%      46.500000
max      58.000000
Name: Age, dtype: float64